<a href="https://colab.research.google.com/github/yf591/sd-model-merge-tool/blob/main/04_Merge_Model_Maker_Ver1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2モデルの単純マージ（Hugging Face用）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output

# Hugging Face Hub, PyTorch, その他必要なライブラリをインストール
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # PyTorchを使用して深層学習モデルを操作します。CUDAバージョン（例: `cu118`）を指定
!pip install diffusers transformers accelerate # Stable Diffusionを扱うための主要ライブラリです。モデルのロードや画像生成の操作を簡素化
!pip install safetensors # 安全かつ軽量なモデル保存形式（`.safetensors`）をサポート
!pip install huggingface-hub # Hugging Face Hubからモデルをダウンロード・管理
!pip install opencv-python # 生成した画像の前処理や後処理に使用
!pip install numpy # 数値計算ライブラリで、モデルや画像の操作に使う
!pip install matplotlib # 生成された画像の可視化に使う
!pip install tqdm # プログレスバーの表示
!pip install optuna # ハイパーパラメータ最適化

output.clear()

In [ ]:
import os
import torch
from safetensors.torch import load_file, save_file
from transformers import AutoConfig, AutoModel
from huggingface_hub import hf_hub_download

In [ ]:
from getpass import getpass

# Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

In [ ]:
#@title ### 関数の定義（設定）

# ヘルパー関数
def download_model(repo_id, filename, token):
    """Hugging Face Hubからモデルをダウンロード"""
    return hf_hub_download(repo_id=repo_id, filename=filename, token=token)

def merge_weights(model1, model2, alpha):
    """指定されたレイヤーで重みをマージする"""
    merged = {}
    for key in model1.keys():
        if key in model2:
            layer_weight = alpha
            try:
                merged[key] = layer_weight * model1[key] + (1 - layer_weight) * model2[key]
            except Exception as e:
                print(f"Skipped key: {key}, Error: {e}")
                merged[key] = model1[key]  # エラーが発生したら model1 の重みをそのまま使う
        else:
            merged[key] = model1[key]
    return merged


def save_merged_model(merged_weights, output_path):
    """マージ済みモデルを保存"""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_file(merged_weights, output_path)

In [ ]:
#@title ### UI設定

#@markdown ### マージするモデルを設定
repo1 = "sazyou-roukaku/chilled_remix" #@param {type:"string"}
file1 = "chilled_remix_v2.safetensors" #@param {type:"string"}
repo2 = "casque/majicmixRealistic_v6" #@param {type:"string"}
file2 = "majicmixRealistic_v6.safetensors" #@param {type:"string"}

#@markdown ### アウトプット先を指定
output_file = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/merged_model_chill_majicmix.safetensors" #@param {type:"string"}

#@markdown ### マージ比率の設定
alpha = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.001}

In [ ]:
#@title ###実行
try:
    print("モデルのダウンロードを開始します...")
    path1 = download_model(repo1, file1, HF_TOKEN)
    path2 = download_model(repo2, file2, HF_TOKEN)

    print("モデルをロード中...")
    model1 = load_file(path1, device="cuda" if torch.cuda.is_available() else "cpu")
    model2 = load_file(path2, device="cuda" if torch.cuda.is_available() else "cpu")

    # ここにモデルのキーを出力するコードを追加
    print("model1 keys:", model1.keys())
    print("model2 keys:", model2.keys())

    print("モデルをレイヤーごとにマージ中...")
    merged_weights = merge_weights(model1, model2, alpha)

    print(f"マージされたモデルを保存します: {output_file}")
    save_merged_model(merged_weights, output_file)
    print("マージ完了！")

except Exception as e:
    print(f"エラーが発生しました: {e}")